In [1]:
# Import libraries
import os
import keras
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import add
from tensorflow.keras import backend as K

from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

print(tf.__version__)

Using TensorFlow backend.


2.1.0


In [2]:
### GPU LIMITING ###

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Get a ResNet50 model
def resnet50_model(classes=4000, *args, **kwargs):
    # Load a model if we have saved one
    if(os.path.isfile('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/resnet_50.h5') == True):
        return keras.models.load_model('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/resnet_50.h5')
    # Create an input layer 
    input = keras.layers.Input(shape=(None, None, 3))
    # Create output layers
    output = keras.layers.ZeroPadding2D(padding=3, name='padding_conv1')(input)
    output = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name='conv1')(output)
    output = keras.layers.BatchNormalization(axis=3, epsilon=1e-5, name='bn_conv1')(output)
    output = keras.layers.Activation('relu', name='conv1_relu')(output)
    output = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='pool1')(output)
    output = conv_block(output, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    output = identity_block(output, 3, [64, 64, 256], stage=2, block='b')
    output = identity_block(output, 3, [64, 64, 256], stage=2, block='c')
    output = conv_block(output, 3, [128, 128, 512], stage=3, block='a')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='b')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='c')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='d')
    output = conv_block(output, 3, [256, 256, 1024], stage=4, block='a')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='b')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='c')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='d')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='e')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='f')
    output = conv_block(output, 3, [512, 512, 2048], stage=5, block='a')
    output = identity_block(output, 3, [512, 512, 2048], stage=5, block='b')
    output = identity_block(output, 3, [512, 512, 2048], stage=5, block='c')
    output = keras.layers.GlobalAveragePooling2D(name='pool5')(output)
    output = keras.layers.Dense(classes, activation='softmax', name='fc1000')(output)
    
    # Create a model from input layer and output layers
    model = keras.models.Model(inputs=input, outputs=output, *args, **kwargs)
    
    # Print model
    print()
    print(model.summary(), '\n')
    
    opt = keras.optimizers.Adam(lr=0.001)
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
   
    # Return a model
    return model

In [5]:
# Create an identity block
def identity_block(input, kernel_size, filters, stage, block):
    
    # Variables
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Create layers
    output = keras.layers.Conv2D(filters1, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(output)
    output = keras.layers.add([output, input])
    output = keras.layers.Activation('relu')(output)
    # Return a block
    return output

In [6]:
# Create a convolution block
def conv_block(input, kernel_size, filters, stage, block, strides=(2, 2)):
    # Variables
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Create block layers
    output = keras.layers.Conv2D(filters1, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(output)
    shortcut = keras.layers.Conv2D(filters3, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input)
    shortcut = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '1')(shortcut)
    output = keras.layers.add([output, shortcut])
    output = keras.layers.Activation('relu')(output)
    # Return a block
    return output


In [ ]:
# Train a model
def train():
    # Variables, 25 epochs so far
    epochs = 20
    batch_size = 128
    train_samples = 380638 # 10 categories with 5000 images in each category
    validation_samples = 8000 # 10 categories with 1000 images in each category
    img_width, img_height = 32, 32
    # Get the model (10 categories)
    model = resnet50_model(4000)
    # Create a data generator for training
    train_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True)
    # Create a data generator for validation
    validation_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        horizontal_flip=True)
    # Create a train generator
    train_generator = train_data_generator.flow_from_directory( 
        'C:/Users/bryan/Desktop/Face recognition dataset/classification_data/train_data', 
        target_size=(img_width, img_height), 
        batch_size=batch_size,
        color_mode='rgb',
        shuffle=True,
        class_mode='categorical')
    # Create a test generator
    validation_generator = validation_data_generator.flow_from_directory( 
        'C:/Users/bryan/Desktop/Face recognition dataset/classification_data/test_data', 
        target_size=(img_width, img_height), 
        batch_size=batch_size,
        color_mode='rgb',
        shuffle=True,
        class_mode='categorical')
    # Start training, fit the model
    model.fit( 
        train_generator, 
        steps_per_epoch=train_samples // batch_size, 
        validation_data=validation_generator, 
        validation_steps=validation_samples // batch_size,
        epochs=epochs)
    # Save model to disk
    model.save('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/resnet_50.h5')
    print('Saved model to disk!')
    # Get labels
    labels = train_generator.class_indices
    # Invert labels
    classes = {}
    for key, value in labels.items():
        classes[value] = key.capitalize()
    # Save classes to file
    with open('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/classes.pkl', 'wb') as file:
        pickle.dump(classes, file)
    print('Saved classes to disk!')
    
    # The main entry point for this module
def main():
    # Train a model
    train()
# Tell python to run main method
if __name__ == '__main__': main()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
_____________________________________________________________________________________________

Found 380638 images belonging to 4000 classes.
Found 8000 images belonging to 4000 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 2973 steps, validate for 62 steps
Epoch 1/20
 374/2973 [==>...........................] - ETA: 7:54 - loss: 9.2233 - accuracy: 3.9689e-04

In [ ]:
# Import libraries
import os
import cv2
import tensorflow.keras
import random
import pickle
import numpy as np

# Evaluate the model
def evaluate():
    
    # Load the model
    model = tensorflow.keras.models.load_model('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/resnet_50.h5')
    
    # Load classes
    classes = {}
    with open('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/classes.pkl', 'rb') as file:
        classes = pickle.load(file)
    
    # Get a list of categories
    categories = os.listdir('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/val_data')
    
    # Get a category a random
    category = random.choice(categories)
    
    # Print the category
    print(category)
    
    # Get images in a category
    images =  os.listdir('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/val_data/' + category)

    # Loop images
    blocks = []
    for i, name in enumerate(images):
        # Limit the evaluation
        if i > 6:
            break;
        # Print the name
        print(name)
        # Get the image
        image = cv2.imread('C:/Users/bryan/Desktop/Face recognition dataset/classification_data/val_data/' + category + '\\' + name)
        # Get input reshaped and rescaled
        input = np.array(image).reshape((1, 64, 64, 3)).astype('float64')/255
        # Get predictions
        predictions = model.predict(input).ravel()
        
        # Print predictions
        print(predictions)
        
        # Get the class with the highest probability
        prediction = np.argmax(predictions)
        
        # Check if the prediction is correct
        correct = True if classes[prediction].lower() == category else False
        
        print("\n")
        print('True' if correct else 'False')
        print(str(round(predictions[prediction] * 100, 2)))
        print("\n")
        
        
        
# The main entry point for this module
def main():
    # Evaluate the model
    evaluate()

# Tell python to run main method
if __name__ == '__main__': main()

In [ ]:
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve

roc_log = roc_auc_score(np.argmax(Y_test, axis=1), np.argmax(prediction, axis=1))
false_positive_rate, true_positive_rate, threshold = roc_curve(np.argmax(Y_test, axis=1), np.argmax(Y_pred_tta, axis=1))
area_under_curve = auc(false_positive_rate, true_positive_rate)

plt.plot([0, 1], [0, 1], 'r--')
plt.plot(false_positive_rate, true_positive_rate, label='AUC = {:.3f}'.format(area_under_curve))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
#plt.savefig(ROC_PLOT_FILE, bbox_inches='tight')
plt.close()